## Ordenemos las ideas

La idea es hacer un clasificador de los títulos con fastText. Para eso, a fastText hay que darle un archivo que tenga en cada fila la etiqueta de la categoría en formato `__label__<CATEGORY>` seguida del texto a clasificado dentro de esa categoría. 

Por otro lado, me gustaría dejar sólo los sustantivos en los títulos de productos y algunas otras cosas como por ejemplo 'PS4', que podríamos reemplazar por la palabra 'consola'.

Entonces, lo que debería hacer en este notebook es:

*   dividir el dataset en dos, uno con los productos en español y otro con los productos en portugués,
*   agregar `__label__` delante de todass las categorías,
*   reemplazar expresiones importantes como nombres particulares de producto por el tipo de producto que es (i.e. PS4 --> consola, por ej.),
*   usar spaCy para determinar qué tipos de palabras hay en cada título y quedarnos con los más importantes.



In [2]:
! curl https://meli-data-challenge.s3.amazonaws.com/train.csv.gz --output train.csv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  659M  100  659M    0     0  81.0M      0  0:00:08  0:00:08 --:--:-- 85.1M


In [0]:
! find . -name '*.csv.gz' -print0 | xargs -0 -n1 gzip -d

In [0]:
import pandas as pd

In [0]:
products = pd.read_csv('train.csv')

In [6]:
products.head()

,title,label_quality,language,category
0,Hidrolavadora Lavor One 120 Bar 1700w Bomba A...,unreliable,spanish,ELECTRIC_PRESSURE_WASHERS
1,Placa De Sonido - Behringer Umc22,unreliable,spanish,SOUND_CARDS
2,Maquina De Lavar Electrolux 12 Kilos,unreliable,portuguese,WASHING_MACHINES
3,Par Disco De Freio Diant Vent Gol 8v 08/ Frema...,unreliable,portuguese,VEHICLE_BRAKE_DISCS
4,Flashes Led Pestañas Luminoso Falso Pestañas P...,unreliable,spanish,FALSE_EYELASHES


Primero agreguemos `__label__` delante de todas las categorías.


In [7]:
products['category'] = products['category'].apply(lambda x : '__label__'+x)
products.head()

,title,label_quality,language,category
0,Hidrolavadora Lavor One 120 Bar 1700w Bomba A...,unreliable,spanish,__label__ELECTRIC_PRESSURE_WASHERS
1,Placa De Sonido - Behringer Umc22,unreliable,spanish,__label__SOUND_CARDS
2,Maquina De Lavar Electrolux 12 Kilos,unreliable,portuguese,__label__WASHING_MACHINES
3,Par Disco De Freio Diant Vent Gol 8v 08/ Frema...,unreliable,portuguese,__label__VEHICLE_BRAKE_DISCS
4,Flashes Led Pestañas Luminoso Falso Pestañas P...,unreliable,spanish,__label__FALSE_EYELASHES


Ahora separo entre títulos en español y títulos en portugués. Después (y para esta primera pruebita) voy a tirar las columnas 'language' y 'label_quality'.

In [0]:
spanish_products = products[products['language'] == 'spanish'].drop(columns=['label_quality', 'language'])
portuguese_products = products[products['language'] == 'portuguese'].drop(columns=['label_quality', 'language'])

In [9]:
spanish_products.head(20)

,title,category
0,Hidrolavadora Lavor One 120 Bar 1700w Bomba A...,__label__ELECTRIC_PRESSURE_WASHERS
1,Placa De Sonido - Behringer Umc22,__label__SOUND_CARDS
4,Flashes Led Pestañas Luminoso Falso Pestañas P...,__label__FALSE_EYELASHES
9,Gatito Lunchera Neoprene,__label__LUNCHBOXES
11,Rosario Contador De Billetes Uv / Mg Detecta F...,__label__BILL_COUNTERS
18,Battery Pack Huawei 13000mah Cuerpo De Metal 2...,__label__PORTABLE_CELLPHONE_CHARGERS
23,Portón De Chapa 3 Hojas Mtr 2.50 Sin Marco,__label__GARAGE_DOORS
24,A4404 Multi Puertos Usb Rápido Cargador Cabeza...,__label__LABEL_MAKERS
26,Base Simil Cemento - 30 Cm X 5 Mm,__label__CAKE_TOPPERS
30,Maquina Para Sublimar Estampar Goldex 60x40,__label__SCREEN_PRINTERS


Pensaba tirar los caracteres que no sean alfanuméricos, pero si hago eso de la forma que sé con regex me tira también las vocales con tilde (y las ñ) y no sé cómo evitarlo (a lo de las ñ sí). De todas formas spaCy detecta los signos de puntuación y ya separa solito las palabras, así que tampoco es que haría falta.

Bueno entonces voy a importar spaCy en español y en portugués.

In [10]:
! python -m spacy download es_core_news_sm

     |████████████████████████████████| 16.2MB 2.6MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-cp36-none-any.whl size=16172936 sha256=b345c75d24f72a6ab27103ead7a247af0190339c5af06904261f41e15e52c824
  Stored in directory: /tmp/pip-ephem-wheel-cache-1v3qd77i/wheels/05/4f/66/9d0c806f86de08e8645d67996798c49e1512f9c3a250d74242
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [11]:
! python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2MB 1.2MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=576a771dbdad4a98fa0accb7708102f96055227ad94f3275c3dd4e3d5846e0ba
  Stored in directory: /tmp/pip-ephem-wheel-cache-uzhm8qbf/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [0]:
import es_core_news_sm as es_spacy
import pt_core_news_sm as pt_spacy

In [0]:
es_nlp = es_spacy.load()
pt_nlp = pt_spacy.load()

Primero voy a ver qué hace con algunos títulos en particular.

In [14]:
doc1 = es_nlp('Notebook Hp Pavilion Dv6500 Special Edition')
print([(w.text, w.pos_) for w in doc1])

[('Notebook', 'PROPN'), ('Hp', 'PROPN'), ('Pavilion', 'PROPN'), ('Dv6500', 'PROPN'), ('Special', 'PROPN'), ('Edition', 'PROPN')]


Aparentemente todo lo que tenga mayúsculas será considerado como nombre propio... ¿o es porque está en inglés? Si es lo primero no me simpatiza porque no quiero que si pongo un adjetivo con mayúscula me lo deje. Y si es lo segundo... todo mal.

In [15]:
doc = es_nlp('Notebook Hp Pavilion Dv6500 Special Edition'.lower())
print([(w.text, w.pos_) for w in doc])

[('notebook', 'PROPN'), ('hp', 'ADP'), ('pavilion', 'PROPN'), ('dv6500', 'PROPN'), ('special', 'PROPN'), ('edition', 'PROPN')]


Bueno claramente el tema es cuando no entiende la palabra pq está en otro idioma, enton le clava 'PROPN'. Pero bueno eso después lo puedo limpiar más adelante, no sé.

Pruebo entonces aplicarle esto a los títulos. PERO hacelo con un set más chico Lisa porque no te da la RAM.

También cambio mayúsculas por minúsculas porque se ve que el modelo aprendió que las cosas con mayúsculas son nombres propios.

In [0]:
small_spanish = spanish_products[0:2000]

In [0]:
import re

dim = '(x?)(\s?)(\d+)(\.?)((\d+)?)(\s?)(\w*)'
dim_2 = dim+'(\s?)(x)(\s?)'+dim
dim_3 = dim_2+'(\s?)(x)(\s?)'+dim

In [86]:
small_spanish['title'] = small_spanish['title'].apply(lambda x : re.sub(dim_3, '', x))
small_spanish['title'] = small_spanish['title'].apply(lambda x : re.sub(dim_2, '', x))
small_spanish['title'] = small_spanish['title'].apply(lambda x : re.sub(dim, '', x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [87]:
small_spanish['title'][26]

'base simil cemento -  '

In [88]:
small_spanish['title'] = small_spanish['title'].apply(lambda x : x.lower())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [89]:
small_spanish['spacy_doc'] = small_spanish['title'].apply(lambda x : es_nlp(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [90]:
print([(w.text, w.pos_) for w in small_spanish['spacy_doc'][4]])

[('flashes', 'PROPN'), ('led', 'PRON'), ('pestañas', 'NOUN'), ('luminoso', 'ADJ'), ('falso', 'ADJ'), ('pestañas', 'ADJ'), ('para', 'ADP'), ('partido', 'NOUN')]


In [0]:
def choose_nouns(doc):
  nouns = []
  for w in doc:
    if w.pos_ in ['NOUN', 'PROPN']:
      nouns.append(w.text)
  return nouns

In [92]:
small_spanish['title_nouns'] = small_spanish['spacy_doc'].apply(choose_nouns)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [95]:
small_spanish = small_spanish.drop(columns='title_trimmed')
small_spanish.head()

,title,category,spacy_doc,title_nouns
0,hidrolavadora lavor one bomba aluminio italia,__label__ELECTRIC_PRESSURE_WASHERS,"(hidrolavadora, lavor, one, , bomba, alumin...","[hidrolavadora, lavor, one, bomba, aluminio, i..."
1,placa de sonido - behringer umc,__label__SOUND_CARDS,"(placa, de, sonido, -, behringer, umc)","[sonido, behringer, umc]"
4,flashes led pestañas luminoso falso pestañas p...,__label__FALSE_EYELASHES,"(flashes, led, pestañas, luminoso, falso, pest...","[flashes, pestañas, partido]"
9,gatito lunchera neoprene,__label__LUNCHBOXES,"(gatito, lunchera, neoprene)","[gatito, lunchera]"
11,rosario contador de billetes uv / mg detecta f...,__label__BILL_COUNTERS,"(rosario, contador, de, billetes, uv, /, mg, d...","[rosario, contador, billetes, uv, mg]"


In [98]:
type(small_spanish['title_nouns'][0][0])

str

In [0]:
def list_to_string(ls):
  str1 = " " 
  return str1.join(ls)

In [0]:
string = list_to_string(small_spanish['title_nouns'][0])

In [101]:
string

'hidrolavadora lavor one bomba aluminio italia'